In [1]:
import  pandas as pd
import numpy as np
df = pd.read_csv('merged_df_abstract_preprocess_all.csv')
df = df.dropna(subset=['processed_abstract'])

C:\Users\cshch\AppData\Local\Temp\ipykernel_18336\2488393254.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('merged_df_abstract_preprocess_all.csv')


In [4]:
print(df.shape)
print(df.columns)

(311557, 20)
Index(['Title', 'Abstract', 'Disciplines', 'Keywords', 'Source',
       'Disciplines_split', 'Astronomy', 'Biology', 'Chemistry',
       'Computer Science', 'Economics', 'Education & instruction',
       'Electrical Engineering', 'European & international law', 'History',
       'Mathematics', 'Physics', 'Social Sciences', 'Statistics',
       'processed_abstract'],
      dtype='object')


In [8]:
# !pip install --upgrade scikit-learn imbalanced-learn

  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.12.2
    Uninstalling imbalanced-learn-0.12.2:
      Successfully uninstalled imbalanced-learn-0.12.2


In [2]:
discipline_mapping = {
    "Astronomy": "Physics",
    "Astronomy/Physics": "Physics",
    "Biology": "Biology",
    "Chemistry/Physics": "Physics",
    "Computer Science": "Computer Science",
    "Computer Science/Electrical Engineering": "Computer Science",
    "Computer Science/Mathematics": "Mathematics",
    "Computer Science/Statistics": "Mathematics",
    "Economics": "Social Sciences",
    "Education & instruction": "Social Sciences",
    "Electrical Engineering": "Electrical Engineering",
    "European & international law": "Social Sciences",
    "History": "Social Sciences",
    "Mathematics": "Mathematics",
    "Mathematics/Physics": "Physics",
    "Mathematics/Statistics": "Mathematics",
    "Physics": "Physics",
    "Social Sciences": "Social Sciences",
    "Statistics": "Mathematics",
    "Computer Science/Electrical Engineering/Mathematics": "Mathematics"
}
# Apply the discipline mapping to the 'Disciplines' column in the DataFrame
df['Disciplines_merged'] = df['Disciplines'].map(discipline_mapping)

# Check the changes
print(df['Disciplines_merged'].value_counts())

Disciplines_merged
Computer Science          133914
Physics                    96677
Mathematics                68196
Social Sciences             5666
Electrical Engineering      5583
Biology                     1521
Name: count, dtype: int64


In [3]:
# Preprocessing
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Vectorize the Text Data using TF-IDF
vectorizer_Logistic_MultiDiscipline = TfidfVectorizer()
X = vectorizer_Logistic_MultiDiscipline.fit_transform(df['processed_abstract'])
y = df['Disciplines_merged']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Oversampling using SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print("Before oversampling:")
print(y_train.value_counts())

print("After oversampling:")
print(pd.Series(y_train_resampled).value_counts())


Before oversampling:
Disciplines_merged
Computer Science          107030
Physics                    77448
Mathematics                54558
Electrical Engineering      4504
Social Sciences             4476
Biology                     1229
Name: count, dtype: int64
After oversampling:
Disciplines_merged
Computer Science          107030
Physics                   107030
Mathematics               107030
Social Sciences           107030
Biology                   107030
Electrical Engineering    107030
Name: count, dtype: int64


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Define the Random Forest model with fixed parameters
model_rf = RandomForestClassifier(
    n_estimators=100,  # Number of trees
    max_depth=None,    # No maximum depth
    random_state=42    # Reproducibility
)

# Train the model on the resampled training data
model_rf.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test set
y_pred_rf = model_rf.predict(X_test)

# Evaluate the model's performance
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Classification Report:\n", classification_report(y_test, y_pred_rf))


Accuracy: 0.8887052253177559
Classification Report:
                         precision    recall  f1-score   support

               Biology       0.94      0.50      0.65       292
      Computer Science       0.88      0.94      0.91     26884
Electrical Engineering       0.45      0.32      0.38      1079
           Mathematics       0.85      0.81      0.83     13638
               Physics       0.96      0.92      0.94     19229
       Social Sciences       0.77      0.63      0.69      1190

              accuracy                           0.89     62312
             macro avg       0.81      0.69      0.73     62312
          weighted avg       0.89      0.89      0.89     62312



In [9]:
# # 3 RandomForest
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score, classification_report
# from sklearn.model_selection import GridSearchCV
#
# # Define the parameter grid for tuning the Random Forest model
# param_grid_rf = {
#     'n_estimators': [100],
#     'max_depth': [None]
#     # 'min_samples_split': [2, 5, 10],
#     # 'min_samples_leaf': [1, 2, 4],
#     # 'bootstrap': [True, False]
# }
#
# # Initialize the Random Forest classifier
# model_rf = RandomForestClassifier(random_state=42)
#
# # Perform grid search with cross-validation
# grid_search_rf = GridSearchCV(
#     estimator=model_rf,
#     param_grid=param_grid_rf,
#     cv=None,  # 5-fold cross-validation
#     n_jobs=-1,  # Utilize all available CPU cores
#     verbose=10  # Display detailed information during the search process
# )
#
# Train the model with resampled training data
# grid_search_rf.fit(X_train_resampled, y_train_resampled)
#
# # Output the best parameters and the best cross-validation score
# print("Best parameters found: ", grid_search_rf.best_params_)
# print("Best cross-validation score: ", grid_search_rf.best_score_)
#
# # Use the best estimator to make predictions on the test set
# best_rf_model = grid_search_rf.best_estimator_
# y_pred_rf = best_rf_model.predict(X_test)
#
# # Evaluate the model's performance
# print("Accuracy:", accuracy_score(y_test, y_pred_rf))
# print("Classification Report:\n", classification_report(y_test, y_pred_rf))


KeyboardInterrupt: 

In [11]:
# Save model
import joblib
import os
save_dir = 'model'
os.makedirs(save_dir, exist_ok=True)

model_file_path = os.path.join(save_dir, 'model_RF_oversampling.joblib')
joblib.dump(model_rf, model_file_path)

# loaded_model = joblib.load(model_file_path)
# print("Model successfully loaded:", loaded_model)

['model\\model_RF_oversampling.joblib']

In [15]:
import pandas as pd

n_top_keywords = 100
feature_names = vectorizer_Logistic_MultiDiscipline.get_feature_names_out()

keywords_dict = {}

for i, label in enumerate(model_Logistic_MultiDiscipline_oversampling.classes_):
    coef = model_Logistic_MultiDiscipline_oversampling.coef_[i]
    top_keywords_idx = coef.argsort()[-n_top_keywords:][::-1]

    top_keywords = [(feature_names[idx], coef[idx]) for idx in top_keywords_idx]
    keywords_dict[label] = top_keywords

keywords_df = pd.DataFrame.from_dict(keywords_dict, orient='index')
keywords_df.to_csv("keywords_from_model/2_keywords_Logistic_oversampling.csv")
